In [1]:
1+1

2

In [2]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader

# OpenAI API 키 설정 (환경변수 또는 직접 입력)
import os
from dotenv import load_dotenv

load_dotenv()

# os.environ["OPENAI_API_KEY"] = ""
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 1. 임베딩 모델 초기화
# OpenAI의 text-embedding-ada-002 모델 사용
# 기본 1536차원 : 1536 × 4바이트 = 6.1KB per 문서
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 1024차원으로 축소: 512 × 4바이트 = 2.0KB per 문서 (약 67% 절약)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", dimensions=1024)


# 2. 샘플 텍스트 데이터 준비
sample_texts = [
    "LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.",
    "벡터 데이터베이스는 텍스트를 수치형 벡터로 변환하여 저장합니다.",
    "RAG는 검색 증강 생성으로, 외부 지식을 활용해 답변을 생성합니다.",
    "임베딩은 텍스트의 의미를 고차원 벡터 공간에 표현하는 기술입니다.",
]

# 3. 벡터 스토어 생성 및 문서 추가
vectorstore = Chroma.from_texts(
    texts=sample_texts,  # 저장할 텍스트 리스트
    embedding=embeddings,  # 사용할 임베딩 모델
    persist_directory="./chroma_db",  # 데이터베이스 저장 경로
)

print("벡터 스토어가 성공적으로 생성되었습니다!")

벡터 스토어가 성공적으로 생성되었습니다!


In [3]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


# 1. 문서 로더 설정 (PDF 파일 예시)
def load_and_process_document(file_path):
    """
    문서를 로드하고 청크로 분할하는 함수
    """
    # 파일 확장자에 따라 적절한 로더 선택
    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    elif file_path.endswith(".txt"):
        loader = TextLoader(file_path, encoding="utf-8")
    else:
        raise ValueError("지원하지 않는 파일 형식입니다.")

    # 문서 로드
    documents = loader.load()

    # 2. 텍스트 분할기 설정
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,  # 각 청크의 최대 크기
        chunk_overlap=200,  # 청크 간 겹치는 부분
        length_function=len,  # 길이 계산 함수
        separators=["\n\n", "\n", " ", ""],  # 분할 기준
    )

    # 문서를 청크로 분할
    chunks = text_splitter.split_documents(documents)

    return chunks


# 사용 예시 (실제 파일이 있을 때)
# chunks = load_and_process_document("sample_document.pdf")
# vectorstore = Chroma.from_documents(
#     documents=chunks,
#     embedding=embeddings,
#     persist_directory="./chroma_db"
# )

In [4]:
chunks = load_and_process_document("sample_document.pdf")
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="./chroma_db"
)

type(vectorstore)


langchain_community.vectorstores.chroma.Chroma

3-3. 유사도 검색 실습

In [5]:
# 1. 기본 유사도 검색
def search_similar_documents(query, k=3):
    """
    쿼리와 유사한 문서를 검색하는 함수

    Args:
        query (str): 검색할 쿼리
        k (int): 반환할 문서 수

    Returns:
        list: 유사한 문서 리스트
    """
    # 유사도 검색 수행
    similar_docs = vectorstore.similarity_search(query=query, k=k)  # 상위 k개 문서 반환

    return similar_docs


# 검색 실행
query = "LangChain이 무엇인가요?"
results = search_similar_documents(query)

print(f"검색 쿼리: {query}")
print("=" * 50)

for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")
    print("-" * 30)

검색 쿼리: LangChain이 무엇인가요?
1. LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.
------------------------------
2. 임베딩은 텍스트의 의미를 고차원 벡터 공간에 표현하는 기술입니다.
------------------------------
3. LS 전선/LS 마린솔루션 
📌미/중 맞설 AI 허브 노리는 사우디, 빈 살만 펀드, 韓 스타트업 쇼핑 
 *HOT 예상 
NAVER/컴퍼니케이/딥노이드/스톤브릿지벤처/폴라리스오피스/플리토/
더존비즈온/솔트룩스 
📌트럼프 휴전 깨졌나. 후티, 홍해 지나던 상선 이틀 연속 공격 
 *HOT 예상 
HMM/흥아해운/KCTC/동방 
 
📈TODAY 종목 Pick 
1️⃣ 삼성물산(초보투자자형) 
✅가격권 150,000 원 ~ 200,000 원 
 
 현재 원전과 주택 양축에서 모두 구조적 전환을 시도. 원전 
부문에서는 기존의 시공 중심 역할에서 벗어나, 
SMR(소형모듈원자로)을 중심으로 한 비즈니스로 확대 
 루마니아 뉴스케일 SMR 프로젝트(24 년 11 월 수주) 시작으로, 
스웨덴 등 북유럽 국가들에서 초기 사업자로서의 입지를 
확보하고자 시도 
 주택 부문에서 넥스트 홈이라는 기술 기반 주거상품을 앞세워 
구조 혁신과 고급화 전략을 동시에 추진하고 있으며 이 시스템이 
핵심 경쟁력으로 작용할 것으로 판단 
 
2️⃣ LS ELECTRIC(안정투자자형) 
✅가격권 246,000 원 ~ 326,000 원
------------------------------


3-4. 점수 기반 검색

In [6]:
# 유사도 점수와 함께 검색
def search_with_scores(query, k=3):
    """
    유사도 점수와 함께 문서를 검색하는 함수
    """
    # 점수와 함께 검색
    results_with_scores = vectorstore.similarity_search_with_score(query=query, k=k)

    return results_with_scores


# 점수 기반 검색 실행
query = "벡터 데이터베이스의 특징"
scored_results = search_with_scores(query)

print(f"검색 쿼리: {query}")
print("=" * 50)

for i, (doc, score) in enumerate(scored_results, 1):
    print(f"{i}. 유사도 점수: {score:.4f}")
    print(f"   내용: {doc.page_content}")
    print("-" * 30)

검색 쿼리: 벡터 데이터베이스의 특징
1. 유사도 점수: 0.4371
   내용: 벡터 데이터베이스는 텍스트를 수치형 벡터로 변환하여 저장합니다.
------------------------------
2. 유사도 점수: 1.1749
   내용: 임베딩은 텍스트의 의미를 고차원 벡터 공간에 표현하는 기술입니다.
------------------------------
3. 유사도 점수: 1.4976
   내용:  미국 시장을 중심으로 AI(인공지능) 데이터센터향 전력기기 
수주가 본격화되고 있는 가운데, 이미 높은 시장점유율을 
확보하고 있는 국내 시장에서도 정부의 AI 인프라 확대 계획에 
힘입어 수혜 기대 
 지난해 말 미국 X 사와 데이터센터용 전력기기 수주 계약을 
체결했고, 최근에는 전력 및 액체냉각 시스템을 구축하는 
버티브(Vertiv)와 파트너십을 체결 
 데이터센터용 전력기기는 기존 양산형 전력기기와 달리 고객의 
요구에 맞춰 개발되는데, LS ELECTRIC 은 빠른 개발 및 인증과 
납기가 큰 경쟁우위로 작용할 것으로 판단 
 
3️⃣ SK 오션플랜트(안정투자자형) 
✅가격권 19,000 원 ~ 25,000 원 
 
 1 분기 실적은 시장 기대치를 상회. 해상풍력 매출 규모 성장에 
비례하여 마진이 정상화되는 과정에 있음 
 연간 매출액 가이던스의 변화가 제한적이기 때문에 수주잔고를 
감안하면 분기 매출액 편차는 크지 않을 것으로 예상 
 다만 올해 연간 매출 대부분을 특수선 부문이 차지할 전망이며 
기존 잔고 소진 이후 국내외 해상풍력 수주 물량 증가에 따른 
매출 Mix 개선이 예상. 국내 재생에너지 정책 제고 기대감으로 
멀티플이 확장되는 국면 
 2030 년까지 서해안 에너지고속도로, 2040 년까지 한반도 
에너지고속도로 건설로 송전 인프라 적기 구축 및 전력망 혁신 
추진 수혜
------------------------------


3-5. 필터링 검색

In [7]:
# 메타데이터를 활용한 필터링 검색
def create_vectorstore_with_metadata():
    """
    메타데이터가 포함된 벡터 스토어 생성
    """
    # 메타데이터가 포함된 문서 데이터
    documents_with_metadata = [
        {
            "text": "LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.",
            "metadata": {"category": "framework", "difficulty": "beginner"},
        },
        {
            "text": "벡터 데이터베이스는 텍스트를 수치형 벡터로 변환하여 저장합니다.",
            "metadata": {"category": "database", "difficulty": "intermediate"},
        },
        {
            "text": "RAG는 검색 증강 생성으로, 외부 지식을 활용해 답변을 생성합니다.",
            "metadata": {"category": "technique", "difficulty": "advanced"},
        },
    ]

    # 텍스트와 메타데이터 분리
    texts = [doc["text"] for doc in documents_with_metadata]
    metadatas = [doc["metadata"] for doc in documents_with_metadata]

    # 메타데이터가 포함된 벡터 스토어 생성
    vectorstore_with_meta = Chroma.from_texts(
        texts=texts,
        metadatas=metadatas,
        embedding=embeddings,
        persist_directory="./chroma_db_with_meta",
    )

    return vectorstore_with_meta


# 필터링 검색 함수
def filtered_search(vectorstore, query, filter_dict, k=3):
    """
    메타데이터 필터를 적용한 검색
    """
    results = vectorstore.similarity_search(
        query=query, k=k, filter=filter_dict  # 필터 조건
    )

    return results


# 필터링 검색 실행 예시
vectorstore_meta = create_vectorstore_with_metadata()

# 초급 난이도 문서만 검색
beginner_results = filtered_search(
    vectorstore_meta, "프레임워크에 대해 알려주세요", {"difficulty": "beginner"}
)

print("초급 난이도 문서 검색 결과:")
for doc in beginner_results:
    print(f"- {doc.page_content}")
    print(f"  메타데이터: {doc.metadata}")

초급 난이도 문서 검색 결과:
- LangChain은 대규모 언어 모델을 활용한 애플리케이션 개발 프레임워크입니다.
  메타데이터: {'category': 'framework', 'difficulty': 'beginner'}
